###Importing necessary libraries

In [1]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors
import numpy as np
import math
import sys
import time
import csv

### Functions

### Function to make a 2D histogram plot
* min and max are determined by x's range, 
* mu and sigma are input estimates of mean and stdev 
* histogram uses nbin bins 
* xlabel is x-axis label (string)
* show = 0 produces figure, show=1 suspends execution to view


In [3]:
#def makeHistogram2D(x, y, mu_x, sigma_x, mu_y, sigma_y, alpha, nbin, show, xlabel, ylabel, addline, alpha): 

# create the histogram of the data
#x, y = np.random.multivariate_normal([mu_x, mu_y], [[sigma_x**2,alpha*sigma_x*sigma_y], [alpha*sigma_x*sigma_y,sigma_y**2]],N).T

#plt.hist2d(x, y, bins=Nbin, range=None, normed=False, weights=None, cmin=None, cmax=None, hold=None)

# add a 'best fit' line
#     y = mlab.normpdf(bins, mu, sigma)
#     plt.plot(bins, y, 'g--')
#    if (addline): 
#        xx = np.linspace(mu-4*sigma, mu+4*sigma, 200)
#        yy = 1/(np.sqrt(2*np.pi)*sigma) * np.exp(-0.5 * ((xx - mu)/sigma)**2)
#        plt.plot(xx, yy, 'r-')
#    plt.xlabel(xlabel)
#    plt.ylabel(ylabel)
#    plt.title('2D Histogram')
    # Tweak spacing to prevent clipping of ylabel
#    plt.subplots_adjust(left=0.15)
#    if (show): 
#        plt.show()
#    else:
#        plt.figure()



### Function to read in {x,y} pair values from a csv file 
* reads in a sample of known size Nsam and return data the (x,y) values as two arrays

In [4]:
def readCSV(filename, Nsam):  

    x = np.zeros(Nsam, float)
    y = np.zeros(Nsam, float)

    print("opening file ", filename)
    with open(filename, 'rb') as f:
        reader = csv.reader(f)
        n = 0
        for row in reader:
            x[n] = float(row[0])
            y[n] = float(row[1])
            n += 1
    print("sample read has size ", n)  # note: no check that n=Nsam here
    return x,y


###MAIN BOOTSTRAP CODE 

In [5]:
if __name__ == "__main__":
    tstart = time.clock()

# read in an existing file of known size
    Nread = 1000000
    x,y = readCSV("14tuesday.csv", Nread)

# downsample x input data if desired  
    Nsam = 1000
    if (Nsam < Nread):
        x = x[0:Nsam]
        print("x downsampled to length ", len(x))
        
# downsample y input data if desired  
    Nsam = 1000
    if (Nsam < Nread):
        y = y[0:Nsam]
        print("y downsampled to length ", len(y))
        

# record x mean and stdev of sample
    xmeanSam = sum(x)/Nsam
    xstdevSam = np.sqrt(sum((x-xmeanSam)**2)/(Nsam-1))
    print ("sample mean and stdev of x", xmeanSam, xstdevSam)
    
# record x mean and stdev of sample
    ymeanSam = sum(y)/Nsam
    ystdevSam = np.sqrt(sum((y-ymeanSam)**2)/(Nsam-1))
    print ("sample mean and stdev of y", ymeanSam, ystdevSam)

# M is the number of bootstrap samples to generate
    M = 1000
    print("Doing", M, "boostrap resamplings on sample of size ", Nsam)

# create arrays to hold iteration values of mean and stdev
    xmean = np.zeros(M,float)
    xstdev = np.zeros(M,float)
    
# create arrays to hold iteration values of mean and stdev
    ymean = np.zeros(M,float)
    ystdev = np.zeros(M,float)
    
# start bootstrap
    i=0
    while(i<M):
        itmp = np.random.random_integers(0, Nsam-1, Nsam)
        xtmp = x[itmp]
        
        jtmp = np.random.random_integers(0, Nsam-1, Nsam)
        ytmp = y[jtmp]

# record mean and stdev for this sample
        xmean[i] = sum(xtmp)/Nsam
        xstdev[i] = np.sqrt(sum((xtmp-xmean[i])**2)/(Nsam-1))
        
        ymean[i] = sum(ytmp)/Nsam
        ystdev[i] = np.sqrt(sum((ytmp-ymean[i])**2)/(Nsam-1))

        if(i%100==0): print("iteration", i)
        i += 1
        
# compute x median and 5, 95%-ile values 
    xmeansort = np.sort(xmean)
    print("5, 50, 95 percent mean = ", xmeansort[int(0.05*M)], xmeansort[int(0.5*M)], xmeansort[int(0.95*M)])
    xstdevsort = np.sort(xstdev)
    print("5, 50, 95 percent stdev = ", xstdevsort[int(0.05*M)], xstdevsort[int(0.5*M)], xstdevsort[int(0.95*M)])
    xsort = np.sort(x)
    print("5, 50, 95 percent stdev = ", xsort[int(0.05*M)], xsort[int(0.5*M)], xsort[int(0.95*M)])
# compute y median and 5, 95%-ile values 
    ymeansort = np.sort(ymean)
    print("5, 50, 95 percent mean = ", ymeansort[int(0.05*M)], ymeansort[int(0.5*M)], ymeansort[int(0.95*M)])
    ystdevsort = np.sort(ystdev)
    print("5, 50, 95 percent stdev = ", ystdevsort[int(0.05*M)], ystdevsort[int(0.5*M)], ystdevsort[int(0.95*M)])
    
    tend = time.clock()
    print("total time elapsed= ", tend-tstart)

# plot frequency distributions from full set of bootstrap resamplings
#    makeHistogram1D(xmean, xmeanSam, xstdevSam/np.sqrt(Nsam), 100, 0, 'mean x', 1)
#     fac = np.sqrt(Nsam*M)
#    makeHistogram1D(xstdev, xstdevSam, xstdevSam/M, 100, 1, 'stdev x', 0)
    
#    makeHistogram2D(xmean, ymean, xmeanSam, xstdevSam/np.sqrt(Nsam), ymeanSam, ystdevSam/np.sqrt(Nsam), 100, 0, 'mean x', 'mean y', 1, np.corrcoef(xmean, ymean)[0, 1])
#      fac = np.sqrt(Nsam*M)
#    makeHistogram2D(xstdev, ystdev, xstdevSam, xstdevSam/M, ystdevSam, ystdevSam/M, 100, 1, 'stdev x', 'stdev y', 0, corrcoef(xstdev, ystdev)[0, 1])

    

opening file  14tuesday.csv


Error: iterator should return strings, not bytes (did you open the file in text mode?)